In [ ]:
import pandas as pd

# Replace 'your_file_path.csv' with the actual path to your CSV file
file_path = '/content/drive/MyDrive/3D Visualization Modification/All_Species_Interaction_Data_CSV/WT_Bas_GSM1671399_01_Rudnerlab_HindIII_HiC_PY79.matrix.csv'
# Read the CSV data into a DataFrame
df = pd.read_csv(file_path)


In [ ]:
df.head()

,Source,Target,interaction_type,Weight
0,1,404,linear1,10000.0
1,1,2,linear1,10000.0
2,2,3,linear1,10000.0
3,3,4,linear1,10000.0
4,4,5,linear1,10000.0


In [ ]:
# Assuming 'df' is the DataFrame containing the edges with their weights
# Filter out edges with a weight of 10000
df_filtered = df[df['Weight'] != 10000]

Top 5% weighted edge for all source node

In [ ]:
# Group by the 'Source' column and calculate the 95th percentile within each group
top_5_percent_thresholds = df_filtered.groupby('Source')['Weight'].quantile(0.95).reset_index()

# Rename the columns for clarity
top_5_percent_thresholds.columns = ['Source', 'WeightThreshold']

# Merge the original dataframe with the thresholds dataframe to compare weights
top_5_percent_edges = pd.merge(df_filtered, top_5_percent_thresholds, on='Source')

# Select edges where the weight is greater than or equal to the threshold
top_5_percent_edges = top_5_percent_edges[top_5_percent_edges['Weight'] >= top_5_percent_edges['WeightThreshold']]

# Drop the threshold column as it is no longer needed
top_5_percent_edges.drop('WeightThreshold', axis=1, inplace=True)

In [ ]:
# Format the edges for JSON output
top_5_percent_edges_formatted = top_5_percent_edges.apply(
    lambda row: {'Source': f"Node{int(row['Source'])}", 'Target': f"Node{int(row['Target'])}", 'Weight': row['Weight']},
    axis=1
)

In [ ]:
top_5_percent_edges_json = top_5_percent_edges_formatted.to_json(orient='records')
with open('/content/drive/MyDrive/3D Visualization Modification/BS_WT_edge.json', 'w') as file:
    file.write(top_5_percent_edges_json)